# Sonification of Bleeding with Bank of Filters

First Meeting (2019-06-11, Sasan and Thomas) @CITEC, TH proposed Filter-bank for feature generation
* The idea is to use a bank of different low-pass filters to create increasingly smooth signals
* these filtered signals serve as source for identifying key moments to anchor sound events
* which then create a multiscale data-driven complex grain structure of the raw instantaneous bleeding data.
* note that the limit of filtering with a cutoff-frequency towards 0 yields the integrated signal.

## Imports

In [ ]:
from scipy import signal
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
import copy
# matplotlib.use('TkAgg')

In [ ]:
#%matplotlib inline
%matplotlib notebook


## Load Data and create filter bank signals

In [ ]:
df = pd.read_csv('log_refactored_correction_factor.csv', na_values=['no info', '.'], delimiter=',')
df_indexed = df.reset_index(drop=False)
index = df_indexed['index']
delta = df_indexed['Delta']
volume = df_indexed['Blood Accumulated']

In [ ]:
N = delta.shape[0]
NF = 15  # number of filters to use
dsf = np.zeros((N, NF))
cfs = np.array(1+np.arange(NF))/(3*NF)  # To do: experiment with exponentially tuned filter frequencies
order = 1 # 2... create oscillations into negative...

for i, cf in enumerate(cfs):
    b, a = signal.butter(order, cf)
    zi = signal.lfilter_zi(b, a)
    z, _ = signal.lfilter(b, a, delta, zi = zi*delta[0])
    dsf[:, i] = copy.copy(z)

# plot the data
#fig = plt.figure(figsize=(15,4))
plt.plot(index, delta, 'k.-', lw=0.2)
plt.plot(dsf, color='r', lw=0.3);
plt.ylim(-0.1,2); plt.xlim(0,800); plt.grid()

## Event-based Sonification of filtered data (min/max/threshold cut-throughs...)

In [ ]:
import sc3nb as scn
sc = scn.startup()

In [ ]:
for i, r in dfs:
    # sonify data here
    pass